### BINANCE MONDAY RANGE

##### Obtener data de Binance
- time, day of week, open, high, low, close

In [2]:
API_KEY = 'I60Y1XmtIjdOI4Gt9xsXJ4gSuslHgNZmERh6hZ1I5fHW1OBTVAoxW6rV0P3KDc03'
SECRET_KEY = 'LbCiOhVchnuAfjM9xUHYnF3iIoJEYhHOJroC0XQ5lFiiLGgm9H8jpFtVjzpB5Aff'
base_url = 'https://fapi.binance.com'

In [64]:
# Codigo para obtener el dia de la semana del df 
from binance.um_futures import UMFutures
from datetime import datetime, timedelta
import pandas as pd
import time

def get_historical_klines(symbol, interval, start_date, end_date=None):
    """ 
    symbol = coin, ej BTCUSDT
    interval = velas, ej 1m, 5m, 1h,
    start_date : Format: YYYY-MM-DD
    end_date : Format: YYYY-MM-DD, default is None = Today
    """
    # Inicializar el cliente de Binance Futures
    client = UMFutures()

    # Convertir las fechas a timestamp en milisegundos
    start_ts = int(datetime.strptime(start_date, "%Y-%m-%d").timestamp() * 1000)
    end_ts = int(datetime.now().timestamp() * 1000) if end_date is None else int(datetime.strptime(end_date, "%Y-%m-%d").timestamp() * 1000) # Default today

    # Inicializar la lista para almacenar todas las velas
    all_klines = []

    # Obtener las velas en bucle
    while start_ts < end_ts:
        # Obtener 1000 velas (máximo permitido por llamada)
        klines = client.klines(symbol=symbol, interval=interval, startTime=start_ts, limit=1000)
        
        if not klines:
            break

        all_klines.extend(klines) #append klines to all_klines
        
        # Actualizar el timestamp de inicio para la siguiente iteración
        start_ts = klines[-1][0] + 1

        # Esperar un poco para evitar alcanzar el límite de la API
        time.sleep(0.1)

    # Convertir los datos a un DataFrame de pandas 
    df = pd.DataFrame(all_klines, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])

    # Convertir el timestamp a datetime
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

    # Convertir las columnas necesarias a tipo float
    df[['open', 'high', 'low', 'close', 'volume']] = df[['open', 'high', 'low', 'close', 'volume']].astype(float)

    # Añadir columna con el día de la semana
    df['day_of_week'] = df['timestamp'].dt.day_name()

    #Escojo las columnas que quiero
    return df[['timestamp', 'open', 'high', 'low', 'close', 'volume', 'day_of_week']]

# Ejemplo de uso
symbol = "BTCUSDT"
interval = "1h"
start_date = "2024-08-04"  # Formato: YYYY-MM-DD

df = get_historical_klines(symbol, interval, start_date)


# Guardar los datos en un archivo CSV
# csv_filename = f"{symbol}_{interval}_from_{start_date.replace('-', '')}.csv"
# df.to_csv(csv_filename, index=False)
# print(f"\nDatos guardados en {csv_filename}")
df

,timestamp,open,high,low,close,volume,day_of_week
0,2024-08-04 05:00:00,60540.6,60915.1,60519.5,60762.0,4013.338,Sunday
1,2024-08-04 06:00:00,60762.1,60840.0,60681.0,60789.0,2121.585,Sunday
2,2024-08-04 07:00:00,60788.9,60801.4,60228.0,60346.0,8237.018,Sunday
3,2024-08-04 08:00:00,60344.6,60649.9,60080.5,60544.6,8546.104,Sunday
4,2024-08-04 09:00:00,60544.6,60710.7,60481.0,60678.8,3813.214,Sunday
...,...,...,...,...,...,...,...
357,2024-08-19 02:00:00,58524.9,58712.5,58383.0,58631.4,4412.073,Monday
358,2024-08-19 03:00:00,58631.5,58736.6,58511.7,58696.6,4141.041,Monday
359,2024-08-19 04:00:00,58696.7,58710.6,58442.4,58548.5,4170.870,Monday
360,2024-08-19 05:00:00,58548.5,58588.5,58339.8,58447.0,4341.214,Monday


### Logica base para el algoritmo

- Iterar el df para determinar el high y low del lunes de la semana
- Añadir este high y low al df
- Por ahora los domingos no se toman en cuenta, ya que las funciones por defecto toman el domingo como si fuera el primer dia de la semana. Cambiar este comportamiento no es facil y genera mucha complicacion innecesaria, ademas no creo que haya muchas oportunidades en la que haga diferencia. BACKLOG.

In [54]:
# este considera el domingo como undefined

def add_monday_high_low(df):
    # Asegurarse de que el índice es un DatetimeIndex
    df = df.set_index('timestamp')
    
    # Inicializar las columnas de high y low del lunes
    df['monday_high'] = float('nan')
    df['monday_low'] = float('nan')
    
    # Obtener el primer lunes en los datos
    first_monday = df.index[df.index.dayofweek == 0][0]
    
    # Iterar sobre las semanas
    for week_start in pd.date_range(start=first_monday, end=df.index[-1], freq='W-MON'):
        week_end = week_start + pd.Timedelta(days=6)
        week_data = df.loc[week_start:week_end]
        
        if not week_data.empty:
            # Obtener los datos solo del lunes
            monday_data = week_data[week_data.index.dayofweek == 0]
            
            if not monday_data.empty:
                monday_high = monday_data['high'].max()
                monday_low = monday_data['low'].min()
                
                # Asignar high y low del lunes a toda la semana, excepto el lunes
                df.loc[week_start+pd.Timedelta(days=1):week_end, 'monday_high'] = monday_high
                df.loc[week_start+pd.Timedelta(days=1):week_end, 'monday_low'] = monday_low
    
    # Reemplazar NaN con 'undefined' para los lunes
    df['monday_high'] = df['monday_high'].apply(lambda x: 'undefined' if pd.isna(x) else x)
    df['monday_low'] = df['monday_low'].apply(lambda x: 'undefined' if pd.isna(x) else x)
    
    # Resetear el índice para que 'timestamp' vuelva a ser una columna
    return df.reset_index()

In [55]:
monday1 = add_monday_high_low(df)

In [56]:
monday1

,timestamp,open,high,low,close,volume,day_of_week,monday_high,monday_low
0,2024-08-11 05:00:00,60861.0,61120.0,60846.6,61080.9,2428.876,Sunday,undefined,undefined
1,2024-08-11 06:00:00,61081.0,61137.4,60967.6,61091.1,2399.512,Sunday,undefined,undefined
2,2024-08-11 07:00:00,61091.1,61233.0,61052.0,61224.9,3501.014,Sunday,undefined,undefined
3,2024-08-11 08:00:00,61225.0,61399.0,61119.4,61226.1,5671.283,Sunday,undefined,undefined
4,2024-08-11 09:00:00,61226.1,62000.0,60960.0,61078.8,16461.738,Sunday,undefined,undefined
...,...,...,...,...,...,...,...,...,...
189,2024-08-19 02:00:00,58524.9,58712.5,58383.0,58631.4,4412.073,Monday,undefined,undefined
190,2024-08-19 03:00:00,58631.5,58736.6,58511.7,58696.6,4141.041,Monday,undefined,undefined
191,2024-08-19 04:00:00,58696.7,58710.6,58442.4,58548.5,4170.870,Monday,undefined,undefined
192,2024-08-19 05:00:00,58548.5,58588.5,58339.8,58447.0,4341.214,Monday,undefined,undefined


In [67]:
#Editando para que salga el domingo con el high dle lunes

def add_monday_high_low(df):
    # Asegurarse de que el índice es un DatetimeIndex
    df = df.set_index('timestamp')
    
    # Inicializar las columnas de high y low del lunes
    df['monday_high'] = float('nan')
    df['monday_low'] = float('nan')
    
    # Obtener el primer lunes en los datos
    first_monday = df.index[df.index.dayofweek == 0][0]
    print(first_monday)
    
    # Iterar sobre las semanas
    for week_start in pd.date_range(start=first_monday, end=df.index[-1], freq='W-MON'):
        week_end = week_start + pd.Timedelta(days=6)
        week_data = df.loc[week_start-pd.Timedelta(days=1):week_end]  # Incluir el domingo anterior
        
        if not week_data.empty:
            # Obtener los datos solo del lunes
            monday_data = week_data[week_data.index.dayofweek == 0]
            
            if not monday_data.empty:
                monday_high = monday_data['high'].max()
                monday_low = monday_data['low'].min()
                
                # Asignar high y low del lunes a toda la semana, incluyendo el domingo anterior, excepto el lunes
                df.loc[week_start-pd.Timedelta(days=1):week_start-pd.Timedelta(seconds=1), 'monday_high'] = monday_high
                df.loc[week_start-pd.Timedelta(days=1):week_start-pd.Timedelta(seconds=1), 'monday_low'] = monday_low
                df.loc[week_start+pd.Timedelta(days=1):week_end, 'monday_high'] = monday_high
                df.loc[week_start+pd.Timedelta(days=1):week_end, 'monday_low'] = monday_low
                
                # Establecer 'undefined' para todas las filas del lunes
                df.loc[week_start:week_start+pd.Timedelta(days=1)-pd.Timedelta(seconds=1), 'monday_high'] = 'undefined'
                df.loc[week_start:week_start+pd.Timedelta(days=1)-pd.Timedelta(seconds=1), 'monday_low'] = 'undefined'
    
    # Resetear el índice para que 'timestamp' vuelva a ser una columna
    return df.reset_index()


In [68]:
monday2 = add_monday_high_low(df)
monday2

2024-08-05 00:00:00


/var/folders/lw/mwc8q8hj6t9bsfnbts70kxth0000gn/T/ipykernel_33696/2085237491.py:35: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'undefined' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[week_start:week_start+pd.Timedelta(days=1)-pd.Timedelta(seconds=1), 'monday_high'] = 'undefined'
/var/folders/lw/mwc8q8hj6t9bsfnbts70kxth0000gn/T/ipykernel_33696/2085237491.py:36: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'undefined' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[week_start:week_start+pd.Timedelta(days=1)-pd.Timedelta(seconds=1), 'monday_low'] = 'undefined'


,timestamp,open,high,low,close,volume,day_of_week,monday_high,monday_low
0,2024-08-04 05:00:00,60540.6,60915.1,60519.5,60762.0,4013.338,Sunday,58286.9,48888.0
1,2024-08-04 06:00:00,60762.1,60840.0,60681.0,60789.0,2121.585,Sunday,58286.9,48888.0
2,2024-08-04 07:00:00,60788.9,60801.4,60228.0,60346.0,8237.018,Sunday,58286.9,48888.0
3,2024-08-04 08:00:00,60344.6,60649.9,60080.5,60544.6,8546.104,Sunday,58286.9,48888.0
4,2024-08-04 09:00:00,60544.6,60710.7,60481.0,60678.8,3813.214,Sunday,58286.9,48888.0
...,...,...,...,...,...,...,...,...,...
357,2024-08-19 02:00:00,58524.9,58712.5,58383.0,58631.4,4412.073,Monday,undefined,undefined
358,2024-08-19 03:00:00,58631.5,58736.6,58511.7,58696.6,4141.041,Monday,undefined,undefined
359,2024-08-19 04:00:00,58696.7,58710.6,58442.4,58548.5,4170.870,Monday,undefined,undefined
360,2024-08-19 05:00:00,58548.5,58588.5,58339.8,58447.0,4341.214,Monday,undefined,undefined
